# 1.处理 并 生成FACE数据
gen_12net_data.py
### 输入:
	WIDER_train/images:人脸图片
	wider_face_train.txt:标注文件,格式:
		impath,[x1,y1,x2,y2]+ //表示一图多张人脸
### 输出:
	DATA/12/positive:包含人脸的图片
	DATA/12/negative:非人脸图片
	DATA/12/part:包含人脸,但是人脸比例不大不小
	DATA/12/pos_12.txt:  DATA/12/positive/0.jpg 1    0.09 -0.19 0.02 -0.05
	DATA/12/neg_12.txt:  DATA/12/negative/0.jpg 0                 //备注:非人脸没必要输出坐标.
	DATA/12/part_12.txt:    DATA/12/part/0.jpg -1    -0.22 -0.28 -0.10 0.08

#### 备注:图片左上角(0,0),右下角(1,1),W=1,H=1,pos/part_12.txt 输出的格式是
	impath,(1/-1),regx1,regy1,regx2,regy2
	regx1,regy1是对左上角(0,0)的修正,regx2,regy2是对右下角(1,1)的修正.

	例如,1 0.09 -0.19 0.02 -0.05:表示存在人脸,人脸在(0.09,-0.19),(1.02,0.95)的box(修正(0,0),(1,1))内 
### 处理过程:
	每次读入条记录:
		1.生成50个负样本(非人脸),大小在 [12,min(w,h)/2]之间,与标注人脸 box IOU<0.3的保存为负样本,
		  更新DATA/12/neg_12.txt,DATA/12/negative.
		2.在标注人脸box左上角随机选取5个size 在 [12,min(w,h)/2]的区域,与标注box IOU<0.3,保存为负样本,
		  更新DATA/12/neg_12.txt,DATA/12/negative.
		3.在标注box附近(box的中心随机移动+-0.2w,+-h),随机选取20个 0.8min(w,h)<=size<=1.25max(w,h)的区域,
			3.1:如果选取的与标注的IOC>0.65,输出为正样本DATA/12/pos_12.txt,DATA/12/positive.
			3.2:如果选取的与标注的0.4<=IOC<=0.65,输出为part样本,更新DATA/12/part_12.txt,DATA/12/part.
### 总结:每读到一张图片,生成55个负例,20(正例+part例).

# 2.处理 并 生成五官数据
    gen_landmark_aug_12.py:
### 输入:
	DATA/trainImageList.txt,人脸的坐标和五官的坐标 
	格式:
	image1_path left,top,right,bottom,p1.x,p1.y,p2.x,p2.y,p3.x,p3.y,p4.x,p4.y,p5.x,p5.y
### 输出:
	DATA/12/train_PNet_landmark_aug:下面经过加强处理的LandMark图片.
	DATA/12/landmark_pnet_aug.txt:输出landmark的标注,格式如下:
		image1_path,2,p1.x,p1.y,p2.x,p2.y,p3.x,p3.y,p4.x,p4.y,p5.x,p5.y
	备注:图片左上角(0,0),右下角(1,1),W=1,H=1,pi.x,pi,y是相对于左上角的坐标!.
### 处理过程:
	每次读入条记录:
		1.输出切割下的人脸到DATA/12/train_PNet_landmark_aug,五官记录到DATA/12/landmark_pnet_aug.txt.
		2.随机把刚才的人脸中心平移(+- 0.2W,+-0.2h),然后取0.8min(w,h)<=size<=1.25max(w,h)的区域,如果这个区域
		和原始的IOC>0.65,把这个区域以及五官(相对于这个区域坐标)分别输出train_PNet_landmark_aug,train_PNet_landmark_aug.txt,这个随机切割迭代10次.
			2.1:有0.5的概率水平反转2生成的图片,并把人脸,五官信息更新到train_PNet_landmark_aug,train_PNet_landmark_aug.txt.
			
            2.2:有0.5的概率旋转2生成的图片(0-+5C),并且反转生成的图片,更新train_PNet_landmark_aug,train_PNet_landmark_aug.txt
			
            2.3:有0.5的概率旋转2生成的图片(-5-0C),并且反转生成的图片,更新train_PNet_landmark_aug,train_PNet_landmark_aug.txt
### 总结:读到一张图片,大约生成 1+10*(1+0.5+0.5*2+0.5*2)=36张人脸.

# 3.合并人脸,五官训练数据
gen_imglist_pnet.py
### 输入:
	DATA/12/pos_12.txt:  DATA/12/positive/0.jpg 1    0.09 -0.19 0.02 -0.05
	DATA/12/neg_12.txt:  DATA/12/negative/0.jpg 0                 //备注:非人脸没必要输出坐标.
	DATA/12/part_12.txt: DATA/12/part/0.jpg -1       -0.22 -0.28 -0.10 0.08
	DATA/12/landmark_pnet_aug.txt
### 输出:
	DATA/imglists/pnet/train_pnet_landmark.txt
### 处理过程:
    在pos,part,landmark数据集中,各选25W个例子,neg数据集中选75W例,按照pos,neg,part,landmark
    的顺序输出DATA/imglists/pnet/train_pnet_landmark.txt.
base_num = 250000
### 总结:生成150W个例子.+,landmark,part各占25W,- 占75W

# 4.生成训练数据TFRecord
gen_PNet_tfrecords.py
### 输入:
	DATA/imglists/pnet/train_pnet_landmark.txt
	DATA/12/positive:包含人脸的图片
	DATA/12/negative:非人脸图片
	DATA/12/part:包含人脸,但是人脸比例不大不小
### 输出:
	DATA/imglists/PNet/train_PNet_landmark.tfrecord_shuffle
### 处理过程:
	把一个numpy表示的属性,序列化后输出成为一条一条TF记录,一个记录表示一个EXAMPLE.
	一个example 4个属性:
		image/encoded: bytes数组 类型,表示图像
		image/label  :int64数组[1],表示数据类型(+,-,landmark,part)
		image/roi    :float 数组[4],表示人脸box
		image/landmark:float 数组[10],表示五官
### 学习:tf.train.Features,tf.train.Feature,tf.train.Example,tf.is.TFRecordWriter的使用.